In [12]:
import findspark
import pickle
import os
findspark.init()

In [5]:
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("location-grouped").setMaster("local[*]")
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)
#spark = SparkSession.builder.appName('Basics').getOrCreate()


In [121]:
masterList = []
for y in range(1900, 2020):
    f = open("./parsed-data/{}-parsed.pkl".format(y), "rb")
    if(os.path.getsize("./parsed-data/{}-parsed.pkl".format(y)) > 0):
        tempList = pickle.load(f)
        masterList = masterList + tempList

In [151]:
len(masterList)

464103

In [159]:
masterRDD = sc.parallelize(tuple(masterList))

missingDataRDD = masterRDD.map(lambda row: row[0]).filter(lambda row: row == None)
locationRDD = masterRDD.map(lambda row: row[3]).filter(lambda row: row != None)
foreignPatents = locationRDD.filter(lambda row: row[1] == "").filter(lambda row: row[2] != "")
domesticPatents = locationRDD.filter(lambda row: row[2] == "US")

totalDomesticPatents = domesticPatents.count()
totalForeignPatents = foreignPatents.count()
totalPatents = totalDomesticPatents + totalForeignPatents #the total patents that have locations.

totalMissingData = missingDataRDD.count()
locationRDD.collect()

[(u'Enns', u'', u'AT'),
 (u'Shanghai', u'', u'CN'),
 (u'Indianapolis', u'IN', u'US'),
 (u'Zwickau', u'', u'DE'),
 (u'Velizy-Villacoublay', u'', u'FR'),
 (u'Somerset', u'NJ', u'US'),
 (u'Nelson', u'', u'NZ'),
 (u'Seongnam-si Gyeonggi-do', u'', u'KR'),
 (u'Biel/Bienne', u'', u'CH'),
 (u'Shanghai', u'', u'CN'),
 (u'Tokyo', u'', u'JP'),
 (u'Basking Ridge', u'NJ', u'US'),
 (u'Somerville', u'MA', u'US'),
 (u'Wermelskirchen', u'', u'DE'),
 (u'Osaka', u'', u'JP'),
 (u'Moreland Hills', u'OH', u'US'),
 (u'Foshan', u'', u'CN'),
 (u'Maitland', u'FL', u'US'),
 (u'Biberach', u'', u'DE'),
 (u'Foshan', u'', u'CN'),
 (u'TEL-AVIV', u'', u'IL'),
 (u'Stuttgart', u'', u'DE'),
 (u'Shanghai', u'', u'CN'),
 (u'Chapel Hill', u'NC', u'US'),
 (u'Yongin-si', u'', u'KR'),
 (u'Singapore', u'', u'SG'),
 (u'Wollerau', u'', u'CH'),
 (u'Boston', u'MA', u'US'),
 (u'Havixbeck', u'', u'DE'),
 (u'Shanghai', u'', u'CN'),
 (u'Yamaga-shi', u'', u'JP'),
 (u'Beijing', u'', u'CN'),
 (u'Wuhan, Hubei', u'', u'CN'),
 (u'Paris', u''

In [160]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

def findByState(state):
    return domesticPatents.filter(lambda row: row[1] == state)

patentsByState = []
for state in states:
    patentsByState.append((state, findByState(state).count()))

In [136]:
stateRDD = sc.parallelize(tuple(patentsByState)).sortBy(lambda row: row[1], False)
stateRDD.collect()

[('CA', 12376),
 ('TX', 3715),
 ('NY', 3060),
 ('MA', 2563),
 ('FL', 2047),
 ('WA', 1795),
 ('NJ', 1732),
 ('IL', 1696),
 ('MI', 1541),
 ('PA', 1496),
 ('OH', 1399),
 ('MN', 1302),
 ('CO', 1035),
 ('NC', 1020),
 ('GA', 935),
 ('VA', 930),
 ('AZ', 918),
 ('OR', 918),
 ('MD', 872),
 ('CT', 815),
 ('WI', 741),
 ('UT', 739),
 ('IN', 670),
 ('MO', 485),
 ('TN', 452),
 ('NV', 378),
 ('SC', 326),
 ('IA', 285),
 ('NH', 283),
 ('LA', 252),
 ('ID', 240),
 ('KS', 233),
 ('AL', 214),
 ('OK', 213),
 ('KY', 204),
 ('NM', 190),
 ('AR', 146),
 ('RI', 146),
 ('DE', 134),
 ('VT', 129),
 ('NE', 109),
 ('DC', 96),
 ('MT', 84),
 ('ME', 78),
 ('MS', 76),
 ('HI', 70),
 ('WV', 60),
 ('WY', 56),
 ('SD', 48),
 ('ND', 43),
 ('AK', 40)]

In [137]:
patentDomesticPercentageRDD = stateRDD.map(lambda row: (row[0], row[1]/float(totalDomesticPatents) * 100))
patentDomesticPercentageRDD.collect()

[('CA', 25.03894632488316),
 ('TX', 7.516134906023024),
 ('NY', 6.190948267141441),
 ('MA', 5.18542497015801),
 ('FL', 4.141461144718474),
 ('WA', 3.6316183462480023),
 ('NJ', 3.504157646630384),
 ('IL', 3.4313229611346023),
 ('MI', 3.1177291763610984),
 ('PA', 3.0266858194913713),
 ('OH', 2.830436805794404),
 ('MN', 2.6341877920974364),
 ('CO', 2.0939972080037226),
 ('NC', 2.06364942238048),
 ('GA', 1.8916786371821068),
 ('VA', 1.8815627086410263),
 ('AZ', 1.8572844801424324),
 ('OR', 1.8572844801424324),
 ('MD', 1.764217937564489),
 ('CT', 1.6488963521961681),
 ('WI', 1.4991806097881726),
 ('UT', 1.49513423837174),
 ('IN', 1.3555344245048253),
 ('MO', 0.9812450684848363),
 ('TN', 0.914479940113703),
 ('NV', 0.7647641977057075),
 ('SC', 0.6595585408784672),
 ('IA', 0.5766079268416048),
 ('NH', 0.5725615554251725),
 ('LA', 0.5098427984704716),
 ('ID', 0.48556456997187775),
 ('KS', 0.47140227001436463),
 ('AL', 0.4329617415582576),
 ('OK', 0.4309385558500415),
 ('KY', 0.4127298844760960

In [138]:
from operator import add
countryCountRDD = foreignPatents.map(lambda row: (row[2], 1)).reduceByKey(add).sortBy(lambda row: row[1], False)
countryCountRDD.collect()

[(u'JP', 11895),
 (u'DE', 5087),
 (u'KR', 4456),
 (u'CA', 2508),
 (u'CN', 2495),
 (u'TW', 2233),
 (u'GB', 2102),
 (u'FR', 1936),
 (u'IL', 1194),
 (u'CH', 867),
 (u'IT', 866),
 (u'NL', 846),
 (u'SE', 814),
 (u'IN', 784),
 (u'AU', 773),
 (u'FI', 412),
 (u'BE', 378),
 (u'DK', 362),
 (u'AT', 358),
 (u'ES', 294),
 (u'SG', 230),
 (u'NO', 200),
 (u'RU', 188),
 (u'IE', 163),
 (u'BR', 159),
 (u'HK', 148),
 (u'NZ', 142),
 (u'MX', 102),
 (u'IR', 93),
 (u'SA', 82),
 (u'PL', 81),
 (u'CZ', 74),
 (u'TR', 74),
 (u'MY', 66),
 (u'HU', 63),
 (u'ZA', 62),
 (u'AR', 48),
 (u'GR', 48),
 (u'PT', 45),
 (u'TH', 45),
 (u'UA', 34),
 (u'CL', 34),
 (u'LU', 29),
 (u'RO', 29),
 (u'CO', 25),
 (u'KW', 25),
 (u'AE', 23),
 (u'SI', 20),
 (u'SK', 16),
 (u'PH', 16),
 (u'EE', 15),
 (u'EG', 14),
 (u'HR', 12),
 (u'BG', 12),
 (u'ID', 12),
 (u'US', 12),
 (u'LT', 11),
 (u'LB', 11),
 (u'IS', 10),
 (u'VN', 9),
 (u'RS', 9),
 (u'JM', 8),
 (u'VE', 8),
 (u'QA', 7),
 (u'PK', 7),
 (u'JO', 7),
 (u'CY', 6),
 (u'AM', 6),
 (u'NG', 6),
 (u'CR

In [139]:
patentCountryDomesticPercentageRDD = countryCountRDD.map(lambda row: (row[0], row[1] / float(totalForeignPatents) * 100))
patentCountryDomesticPercentageRDD.collect()

[(u'JP', 27.460350439781152),
 (u'DE', 11.743657224646213),
 (u'KR', 10.286954313548954),
 (u'CA', 5.789874645058522),
 (u'CN', 5.759863333102477),
 (u'TW', 5.155019969065263),
 (u'GB', 4.852598287046656),
 (u'FR', 4.469376918992543),
 (u'IL', 2.7564235750398227),
 (u'CH', 2.0015236512223837),
 (u'IT', 1.9992150887642266),
 (u'NL', 1.9530438396010805),
 (u'SE', 1.8791698409400468),
 (u'IN', 1.8099129671953276),
 (u'AU', 1.7845187801555968),
 (u'FI', 0.9511277327608099),
 (u'BE', 0.8726366091834614),
 (u'DK', 0.8356996098529446),
 (u'AT', 0.8264653600203153),
 (u'ES', 0.6787173626982478),
 (u'SG', 0.5309693653761802),
 (u'NO', 0.4617124916314611),
 (u'RU', 0.4340097421335734),
 (u'IE', 0.3762956806796408),
 (u'BR', 0.3670614308470116),
 (u'HK', 0.3416672438072812),
 (u'NZ', 0.3278158690583374),
 (u'MX', 0.23547337073204516),
 (u'IR', 0.2146963086086294),
 (u'SA', 0.18930212156889906),
 (u'PL', 0.18699355911074175),
 (u'CZ', 0.1708336219036406),
 (u'TR', 0.1708336219036406),
 (u'MY', 0.1

In [112]:
totalForeignPatents / float(totalPatents) * 100

46.705986371086

In [120]:
float(totalPatents) / totalMissingData * 100

25.27235965098725